In [2]:
from clipper_admin import ClipperConnection, DockerContainerManager
clipper_conn = ClipperConnection(DockerContainerManager())
clipper_conn.start_clipper()

20-01-12:21:45:19 INFO     [docker_container_manager.py:184] [default-cluster] Starting managed Redis instance in Docker
20-01-12:21:45:23 INFO     [docker_container_manager.py:276] [default-cluster] Metric Configuration Saved at /tmp/tmpje6skfad.yml
20-01-12:21:45:24 INFO     [clipper_admin.py:162] [default-cluster] Clipper is running


In [3]:
clipper_conn.register_application(name="hello-world", input_type="doubles", default_output="-1.0", slo_micros=100000)

20-01-12:21:45:53 INFO     [clipper_admin.py:236] [default-cluster] Application hello-world was successfully registered


In [4]:
clipper_conn.get_all_apps()

['hello-world']

In [5]:
def feature_sum(xs):
    return [str(sum(x)) for x in xs]

In [6]:
from clipper_admin.deployers import python as python_deployer

In [7]:
python_deployer.deploy_python_closure(clipper_conn, name="sum-model", version=1, input_type="doubles", func=feature_sum)

20-01-12:21:49:26 INFO     [deployer_utils.py:41] Saving function to /tmp/tmphl1z9a_1clipper
20-01-12:21:49:26 INFO     [deployer_utils.py:51] Serialized and supplied predict function
20-01-12:21:49:26 INFO     [python.py:192] Python closure saved
20-01-12:21:49:26 INFO     [python.py:206] Using Python 3.6 base image
20-01-12:21:49:26 INFO     [clipper_admin.py:534] [default-cluster] Building model Docker image with model data from /tmp/tmphl1z9a_1clipper
20-01-12:21:49:27 INFO     [clipper_admin.py:539] [default-cluster] Step 1/2 : FROM clipper/python36-closure-container:0.4.1
20-01-12:21:49:27 INFO     [clipper_admin.py:539] [default-cluster]  ---> e5b9dc250c05
20-01-12:21:49:27 INFO     [clipper_admin.py:539] [default-cluster] Step 2/2 : COPY /tmp/tmphl1z9a_1clipper /model/
20-01-12:21:49:27 INFO     [clipper_admin.py:539] [default-cluster]  ---> 00fe777d67f0
20-01-12:21:49:27 INFO     [clipper_admin.py:539] [default-cluster] Successfully built 00fe777d67f0
20-01-12:21:49:27 INFO   

In [8]:
clipper_conn.link_model_to_app(app_name="hello-world", model_name="sum-model")

20-01-12:21:49:47 INFO     [clipper_admin.py:303] [default-cluster] Model sum-model is now linked to application hello-world


In [9]:
import requests, json, numpy as np
headers = {"Content-type": "application/json"}
requests.post("http://localhost:1337/hello-world/predict", headers=headers, data=json.dumps({"input": list(np.random.random(10))})).json()

{'query_id': 1, 'output': 5.217010286187675, 'default': False}

In [11]:
clipper_conn.stop_all()

20-01-13:06:40:29 INFO     [clipper_admin.py:1424] [default-cluster] Stopped all Clipper cluster and all model containers
